<a href="https://colab.research.google.com/github/liuguliu/study_notebook/blob/main/Megatron_LM%E7%AC%94%E8%AE%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from megatron.core import mpu, tensor_parallel

mpu.initialize_model_parallel(args.tensor_model_parallel_size,
            args.pipeline_model_parallel_size,
            args.virtual_pipeline_model_parallel_size,
            args.pipeline_model_parallel_split_rank)

In [ ]:
'''colossalai 2D张量并行'''

import colossalai
import colossalai.nn as col_nn
import torch
from colossalai.utils import print_rank_0
from colossalai.context import ParallelMode
from colossalai.core import global_context as gpc
from colossalai.utils import get_current_device

# 并行设置
CONFIG = dict(parallel = dict(
    data = 1,
    pipeline = 1,
    tensor = dict(size=4, mode='2d'),
))

parser = colossalai.get_default_parser()

colossalai.launch(config = CONFIG,
      rank = args.rank,
      world_size = args.world_size,
      local_rank = rags.local_rank,
      host = args.host,
      port = args.port)

class MLP(torch.nn.Module):
  def __init__(self, dim: int = 256):
    super().__init__()
    intermediate_dim = dim * 4
    self.dense_1 = col_nn.Linear(dim, intermediate_dim)
    print_rank_0(f'Weight of the first linear layer: {self.dense_1.weight.shape}')
    self.activation = torch.nn.GELU()
    self.dense_2 = col_nn.Linear(intermediate_dim, dim)
    print_rank_0(f'Weight of the second linear layer: {self.dense_2.weight.shape}')
    self.dropout = col_nn.Dropout(0.1)

    def forward(self, x):
      x = self.dense_1(x)
      print_rank_0(f'Output of the first linear layer: {x.shape}')
      x = self.activation(x)
      x = self.dense_2(x)
      print_rank_0(f'Output of the second linear layer: {x.shape}')
      x = self. dropout(x)
      return x


# 创建模型
m = MLP()

# 随机输入一些数据来运行这个模型
x = torch.randn((16,256), device = get_current_device())

# partition input
torch.distributed.broadcast(x, src = 0)
x = torch.chunk(x, 2, dim=0)[gpc.get_local_rank(ParallelMode.PARALLEL_2D_COL)]
x = torch.chunk(x, 2, dim=-1)[gpc.get_local_rank(ParallelMode.PARALLEL_2D_ROW)]
print_rank_0(f'Input: {x.shape}')

x = m(x)


In [ ]:
'''colossalai 2.5D张量并行'''

# 并行设置
CONFIG = dict(parallel = dict(
    data = 1,
    pipeline = 1,
    tensor = dict(size=8, mode='2.5d', depth=2),
))

# 创建模型
m = MLP()

# 随机输入一些数据来运行这个模型
x = torch.randn((16,256), device=get_current_device())

# partition input
torch.distributed.broadcast(x, src=0)
x = torch.chunk(x, 2, dim=0)[gpc.get_local_rank(ParalleMode.PARALLEL_2P5D_DEP)]
x = torch.chunk(x, 2, dim=0)[gpc.get_local_rank(ParallelMode.PARALLEL_2P5D_COL)]
x = torch.chunk(x, 2, dim=-1)[gpc.get_local_rank(ParallelMode.PARALLEL_2P5D_ROW)]
print_rank_0(f'Input:{x.shape}')

x = m(x)

In [ ]:
'''colossalai 3D张量并行'''

# 并行设置
CONFIG = dict(parallel = dict(
    data = 1,
    pipeline = 1,
    tensor = dict(size=8, model='3d'),
))

# 创建模型
m = MLP()

# 随机输入一些数据来运行这个模型
x = torch.randn((16, 256), device = get_current_device())

# partition input
torch.distributed.broadcast(x, src = 0)
x = torch.chunk(x, 2, dim=0)[gpc.get_local_rank(ParallelMode.PARALLEL_3D_WEIGHT)]
x = torch.chunk(x, 2, dim=0)[gpc.get_local_rank(ParallelMode.PARALLEL_3D_INPUT)]
x = torch.chunk(x, 2, dim=-1)[gpc.get_local_rank(ParallelMode.PARALLEL_3D_OUTPUT)]
print_rank_0(f'Input: {x.shape}')

x = m(x)

In [ ]:
'''pytorch张量并行'''

from torch.distributed._tensor import DeviceMesh
from torch.distributed.tensor.parallel import PairwiseParallel, parallelize_module

# 通过设备网络根据给定的 world_size 创建分片计划
device_mesh = DeviceMesh("cuda", torch.arange(0, args.world_size))

# 创建模型并移动到GPU
model = ToyModel().cuda(rank)

# 为并行化模块创建优化器
LR = 0.25
optimizer = torch.optim.SGD(model.parameters(), lr = LR)

# 根据给定的并行风格并行化模块（这里指定为PairwiseParallel），
# colwise 和 rowwise 样式串联为固定对，就像【Megatron-LM】(https://arxiv.org/abs/1909.08053)所做的那样。
model = parallellize_module(model, device_mesh, PairwiseParallel())

# 对分片模块执行多次前向/后向传播和优化器对参数进行更新
for i in range(args.iter_nums):
  # 对于 TP, 所有 TP rank的输入需要相同。
  # 设置随机种子是为了模块数据加载器的行为。
  if rank==0:
    print(f"-----------{i}-----------------")
  torch.manual_seed(i)
  inp = torch.rand(20,10).cuda(rank)
  if rank==0:
    print(f"rank: {rank}, input shape: {inp.shape}")
  output = model(inp)
  if rank==0:
    print(f"rank: {rank}, input shape: {output.shape}")
  output.sum().backward()
  optimizer.step()

